In [ ]:
import pandas
import bokeh
from bokeh.io import show
from bokeh.layouts import layout
from bokeh.io import output_notebook
from bokeh.io import output_file
from bokeh.models import EqHistColorMapper
from bokeh.models import ColumnDataSource
from bokeh.models import CustomJS
from bokeh.models import HoverTool
from bokeh.models import Slider
from bokeh.palettes import Viridis8 as palette
palette = tuple(reversed(palette))

from bokeh.plotting import figure

bokeh.sampledata.download()
from bokeh.sampledata.us_counties import data as counties

In [ ]:
autism = pandas.read_csv("ADV_AllData.csv")
state_list = [s.lower() for s in set(autism['Location'])]
state_list.remove('ak')
state_list.remove('hi')

census = pandas.read_csv("census.txt", dtype={'year':int, 'fips':str, 'age':int, 'pop':int})
tot_pop = census.groupby(['year', 'fips'])
maxim = census.groupby(['year', 'fips'])['pop'].sum().max()
minim = census.groupby(['year', 'fips'])['pop'].sum().min()

In [ ]:
counties = {
    code: county for code, county in counties.items() if county["state"] in state_list
}
county_pop = {}
for i in range(18):
    county_pop[2000+i] = []
    for county_id in counties:
        curr_fips = "%02d"%(county_id[0]) + "%03d"%(county_id[1])
        if (2000+i, curr_fips) in tot_pop.groups:
            county_pop[2000+i] += [float(tot_pop.get_group((2000+i,curr_fips))['pop'].sum())]
        else:
            county_pop[2000+i] += [0.]

In [ ]:
output_notebook()



county_xs = [county["lons"] for county in counties.values()]
county_ys = [county["lats"] for county in counties.values()]

county_names = [county['name'] for county in counties.values()]
color_mapper = EqHistColorMapper(palette=palette, high=maxim, low=minim, low_color="#FFFFFF")



data = dict(x=county_xs,
            y=county_ys,
            name=county_names)
for i in range(18):
    data.update({"pop_%d"%(2000+i) : county_pop[2000 + i]})
data.update({"pop" : county_pop[2000]})

source = ColumnDataSource(data)

TOOLS = "pan,wheel_zoom,reset,hover,save"

p = figure(plot_width=1000, plot_height=800,
    title="U.S. population per county", tools=TOOLS,
    x_axis_location=None, y_axis_location=None,
    tooltips=[
        ("Name", "@name"), ("Population", "@pop"), ("(Long, Lat)", "($x, $y)")
    ])
p.grid.grid_line_color = None
p.hover.point_policy = "follow_mouse"

patch_renderer = p.patches('x', 'y', source=source,
                           fill_color={'field': 'pop', 'transform': color_mapper},
                           fill_alpha=0.7, line_color="white", line_width=0.5)

code = """
console.log('slider: value=' + this.value, this.toString())
var data = source.data;
var pop = data['pop']
var newpop = data['pop_' + this.value]
for (var i = 0; i < pop.length; i++) {
    pop[i] = newpop[i]
}
source.change.emit();
"""
callback = CustomJS(args=dict(source=source), code=code)
slider = Slider(
    start=2000,
    end=2017,
    value=2000,
    step=1,
    title="Year"
)
slider.js_on_change("value", callback)


show(layout([[p], [slider]], sizing_mode='scale_width'))
output_file("US_pop.html")
